# Prep OpenAI embeddings

このノートブックは、Azure OpenAI のモデルを使用して、一般的な日本語の単語の集合に対する Embeddings を生成します。

In [ ]:
import csv
import logging
import json
import os

from azure.identity import AzureCliCredential, get_bearer_token_provider
import openai
import dotenv

logging.basicConfig(level=logging.INFO)
dotenv.load_dotenv()

# Set up OpenAI client
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_EMBEDDING_DEPLOYMENT = os.getenv("AZURE_OPENAI_EMBEDDING_DEPLOYMENT")
AZURE_OPENAI_EMBEDDING_DIMENSIONS = os.getenv("AZURE_OPENAI_EMBEDDING_DIMENSIONS") 
subscription_key = os.getenv("AZURE_OPENAI_API_KEY")  

azure_credential = AzureCliCredential()
token_provider = get_bearer_token_provider(azure_credential, "https://cognitiveservices.azure.com/.default")
openai_client = openai.AzureOpenAI(
    api_version="2023-05-15",
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    #azure_ad_token_provider=token_provider,
    api_key=subscription_key,  
)

# 単一の埋め込みリクエストで入力の配列を送信する場合、配列の最大サイズは 2048 です。
def get_embeddings(words):
    """OpenAI を使用してバッチ（すべての単語を一度に）で埋め込みを計算する"""
    word_vectors = {}
    dimensions_args = (
        {"dimensions": int(AZURE_OPENAI_EMBEDDING_DIMENSIONS)}
        if AZURE_OPENAI_EMBEDDING_DIMENSIONS
        else {}
    )

    embeddings_response = openai_client.embeddings.create(
        model=AZURE_OPENAI_EMBEDDING_DEPLOYMENT,
        input=words,
        **dimensions_args
    )
    for word, embedding_object in zip(words, embeddings_response.data):
        word_vectors[word] = embedding_object.embedding
    return word_vectors

print("AZURE_OPENAI_EMBEDDING_DEPLOYMENT: ", AZURE_OPENAI_EMBEDDING_DEPLOYMENT)
print("AZURE_OPENAI_EMBEDDING_DIMENSIONS: ", AZURE_OPENAI_EMBEDDING_DIMENSIONS)

In [ ]:
# busho_nouns_neologd_1000.csv を開き、最初の列を単語として読み取ります。
words = []
with open('embeddings/busho_nouns_neologd_1000.csv') as f:
    reader = csv.reader(f)
    next(reader)
    for row in reader:
        words.append(row[0])

print(len(words), words[:10])

In [ ]:
# すべての単語の埋め込みを計算する
word_vectors = get_embeddings(words)
print(len(word_vectors))

In [ ]:
# 埋め込みをファイルに保存する
filename = f"embeddings/words_{AZURE_OPENAI_EMBEDDING_DEPLOYMENT}-{AZURE_OPENAI_EMBEDDING_DIMENSIONS}.json"

with open(filename, 'w') as f:
    json.dump(word_vectors, f, indent=4, ensure_ascii=False)
